In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
tra2 = pd.read_csv("./DataSet/momo_trans (2).csv")
tra2.head()

,timestamp,user_id,service_id,store_id,amount,time,hour,dayofweek,category,dayofweek_number
0,1.564590e+12,8.160000e+18,1.330000e+18,3.210000e+18,85000,31/07/2019 16:20,17,Wednesday,cvs,2
1,1.564600e+12,2.330000e+18,1.560000e+18,5.010000e+18,996000,31/07/2019 19:06,18,Wednesday,mass merchant,2
2,1.564600e+12,6.330000e+18,2.840000e+18,6.240000e+18,190000,31/07/2019 19:06,18,Wednesday,mass merchant,2
3,1.564600e+12,4.730000e+18,2.840000e+18,6.240000e+18,961000,31/07/2019 19:06,18,Wednesday,mass merchant,2
4,1.564600e+12,3.580000e+18,5.250000e+18,8.850000e+18,1278000,31/07/2019 19:06,20,Wednesday,mass merchant,2


In [3]:
listCategory = tra2['category'].unique()
listCategory

array(['cvs', 'mass merchant', 'coffee chains and milk tea', 'restaurant',
       'fast food', 'shopping', 'mass ecom', 'supermarket'], dtype=object)

In [ ]:
percentage = {}
for i in listCategory:
    percentage[i] = (tra2["category"] == i).sum() / tra2.shape[0] * 100
percentage

{'cvs': 29.560135617416133,
 'mass merchant': 35.992445871996196,
 'coffee chains and milk tea': 19.100741533825047,
 'restaurant': 5.75025775239908,
 'fast food': 4.31011579030851,
 'shopping': 1.140653501467206,
 'mass ecom': 0.14870330716155128,
 'supermarket': 3.9969466254262827}

In [ ]:
X = tra2[["hour","dayofweek_number"]]
y = tra2["category"]
X.sample(10)

,hour,dayofweek_number
684327,4,3
717676,5,5
192258,12,1
683385,11,5
971857,5,0
367651,11,0
848757,1,0
454053,13,5
92028,8,6
5265,12,6


In [ ]:
y = pd.get_dummies(y)
y.head()

,coffee chains and milk tea,cvs,fast food,mass ecom,mass merchant,restaurant,shopping,supermarket
0,0,1,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
training_data = pd.concat ([X_train,y_train],axis = 1)
training_data.head()

In [ ]:
test_data = pd.concat ([X_test,y_test],axis = 1)
test_data.head()

In [ ]:
data_value_counts = {}
for i in listCategory:
    data_value_counts[i] = training_data[i].value_counts()[1]
data_value_counts = pd.Series(data_value_counts)
data_value_counts

# Under sample data

In [ ]:
min_data = data_value_counts.min()

under_sample = pd.DataFrame()
for cat in data_value_counts.index:
    data = training_data[training_data[cat] == 1].sample(min_data)
    under_sample = pd.concat([under_sample, data], axis=0)

for cat in data_value_counts.index:
    cat_data = under_sample[under_sample[cat] == 1]
    print("Percentage of " + cat,cat_data.shape[0] / under_sample.shape[0])

under_sample.sample(10)

# Dự đoán từng cái category có hay không

In [ ]:
for i in listCategory:
    print(i)
    print(training_data[i].value_counts())
    print("=========")

In [ ]:
# Under sample ở từng category có hay không
def under_sample(cat):
    min = training_data[cat].value_counts()[1]
    
    under_sample = pd.concat([
        training_data[training_data[cat] == 0].sample(min),
        training_data[training_data[cat] == 1].sample(min)],
        axis=0
    )
    
    X_under = under_sample[["hour","dayofweek_number"]]
    y_under = under_sample[cat]
    return {"X": X_under, "y": y_under}

In [ ]:
dict_under_sample = {}
for cat in listCategory:
    dict_under_sample[cat] = under_sample(cat)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
dictmodel = {}
for cat in listCategory:
#     model = LogisticRegression() # 1 (0.49)
#     model = DecisionTreeClassifier() # 2 (0.39, 0.44)
    model =  RandomForestClassifier(n_estimators=50, random_state=101, n_jobs=-1)
#     model = SVC()
#     model = GaussianNB()
#     model = BernuolliNB()
    model.fit(dict_under_sample[cat]["X"],dict_under_sample[cat]["y"])
    dictmodel[cat] = model

In [ ]:
Y_pred = {}
for cat in listCategory:
    Y_pred[cat] = dictmodel[cat].predict(X_test)
Y_pred

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
for cat in listCategory:
    print(cat)
    print(classification_report(test_data[cat],Y_pred[cat]))

In [ ]:
import pickle
import os

for modelcate in dictmodel:
    pickle.dump(dictmodel[modelcate], open(os.path.join('TrainModel', modelcate + '.pkl'),'wb'), protocol=4)

# Read model and Predict

In [ ]:
def ReadModel(catname):
    mymodel = None
    try:
        with open("./TrainModel/"+catname+".pkl", 'rb') as model:
            mymodel = pickle.load(model)
    except IOError:
        print("File not found!!")
    return mymodel

In [ ]:
dictmodel = {}
for cat in listCategory:
    dictmodel[cat] = ReadModel(cat)


In [ ]:
param = pd.DataFrame(np.array([[8,7]]), columns=['hour','dayofweek_number'])
re = [cat for cat in listCategory if dictmodel[cat].predict(param)[0] == 1]
re


In [ ]:
import operator
pred = {}
for cat in listCategory:
    neg , pos = dictmodel[cat].predict_proba(param)[0]
    if pos > neg:
        pred[cat] = pos
sorted(pred.items(), key=operator.itemgetter(0))